### Prepare Repo

In [1]:
# !pip install omegaconf==2.1.1
# !pip install hydra-core==1.1.1
# !pip install -U numpy==1.23.5
# !apt-get update && apt-get install -y python3-opencv
# !pip install opencv-python
# !pip install scikit-image 
# !pip install transformers
# !pip install datasets
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install wandb
# !pip install scikit-learn

In [2]:
# !git clone https://github.com/facebookresearch/av_hubert.git

# %cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

# %cd fairseq
# !pip install ./

In [3]:
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from transformers import Trainer, TrainingArguments
import sys
sys.path.append("/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert")
%cd /home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import wandb
from torch.utils.data import Dataset, Subset
import os
import datetime

/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert
3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]


In [4]:
print(device)
!nvidia-smi

cuda
Sun Oct 22 09:34:31 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:53:00.0 Off |                  N/A |
|  0%   46C    P8              44W / 300W |      5MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

### Download Model

In [5]:
# os.makedirs("/home/check_pts/")
# !wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O /home/check_pts/avhubert.pt

### Build Model Pipeline

In [6]:
user_dir = "/home/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "/home/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Checkpoint: fine-tuned


### Load Dataset

In [7]:
import torch
print(torch.__version__)

2.1.0


In [8]:
from avhubert_ds import AVHUBERTDataset
mmser_ds = torch.load("/home/avhubert_ds2.pt")
mmser_ds.video_path = "/home/face_raw/"

# outputs = model.extract_finetune(mmser_ds[:2])

In [9]:
mmser_ds.cached = False
mmser_ds.__cache__()

100%|██████████| 5531/5531 [08:25<00:00, 10.95it/s]


### Define the model

In [10]:
from avhubert_classifier import AVHUBERTClassifier

In [11]:
classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
classifier(**mmser_ds[:4])

{'logits': tensor([[-0.0373,  0.0067,  0.1036, -0.1225],
         [-0.0195,  0.0183,  0.0853, -0.1143],
         [-0.0762,  0.0041,  0.1067, -0.1074],
         [-0.0652, -0.0080,  0.1014, -0.1166]], grad_fn=<AddmmBackward0>)}

### Build Train Test DS

In [12]:
meta_df_ = mmser_ds.df_
sess_dict = mmser_ds.df_.groupby("session").groups
all_indices = set(mmser_ds.df_.index.tolist())

sess_ds = {}
for sess in sess_dict:
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])

In [13]:
def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

### Run Pipeline

API: 2999b8f99f0f62b4f64c48a1c8be9a16945183e9

In [14]:
import json
sess_id = list(sess_dict.keys())[7]
print("="*10, sess_id, "="*10)

avhubert_classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
for param in avhubert_classifier.encoder.parameters():
    param.requires_grad = False

wandb.init()
print(sess_id)
train_set, val_set, test_set = build_ds(sess_id)


========== Ses04M ==========


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mmser. Use `wandb login --relogin` to force relogin


Ses04M
Train Samples: 3789
Val Samples: 1264
Test Samples: 478


In [15]:
output_dir=os.path.join("check_pts", "AVHUBERT", sess_id, datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=24
training_args.per_device_eval_batch_size=24
training_args.logging_steps = int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=7e-4
training_args.num_train_epochs=50

In [ ]:
from avhubert_trainer import CustomTrainer , compute_metrics
avhubert_classifier = avhubert_classifier.to(device)
trainer = CustomTrainer(
    model=avhubert_classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Wa,Ua,F1,Accuracy
41,1.334300,1.296179,0.370253,0.373608,0.301514,0.370253
82,1.288000,1.255270,0.382911,0.329396,0.311365,0.382911
123,1.235400,1.197959,0.469937,0.453114,0.418037,0.469937
164,1.218100,1.175517,0.472310,0.441990,0.435606,0.472310
205,1.184600,1.181879,0.462025,0.439062,0.446511,0.462025
246,1.180800,1.164788,0.496835,0.496915,0.478611,0.496835
287,1.154100,1.146474,0.486551,0.474995,0.459160,0.486551
328,1.138400,1.128039,0.510285,0.514040,0.505770,0.510285
369,1.152800,1.146407,0.507120,0.502080,0.480332,0.507120
410,1.143000,1.143006,0.500791,0.531895,0.493888,0.500791


/home/multi_modal_ser/finetune_encoder/audio_video/avhubert_trainer.py:34: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_f1 = load_metric("f1")


In [ ]:
eval_result = trainer.evaluate()
test_result = trainer.predict(test_set).metrics

In [ ]:
FREEZE_PROJ_PATH = "/home/freeze/{}/projector".format(sess_id)
FREEZE_CLAS_PATH = "/home/freeze/{}/classifier".format(sess_id)
os.makedirs(FREEZE_PROJ_PATH, exist_ok=True)
os.makedirs(FREEZE_CLAS_PATH, exist_ok=True)

FREEZE_PROJ = os.path.join(FREEZE_PROJ_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")
FREEZE_CLAS = os.path.join(FREEZE_CLAS_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")

torch.save(avhubert_classifier.projector.state_dict(), FREEZE_PROJ)
torch.save(avhubert_classifier.classifier.state_dict(), FREEZE_CLAS)

model.projector.load_state_dict(torch.load(FREEZE_PROJ))
model.classifier.load_state_dict(torch.load(FREEZE_CLAS))

print(eval_result)
print(test_result)

json_test = json.dumps(test_result, indent=4)
json_eval = json.dumps(eval_result, indent=4)

# Writing to sample.json
with open("{}_eval.json".format(sess_id), "w") as outfile:
    outfile.write(json_eval)
with open("{}_test.json".format(sess_id), "w") as outfile:
    outfile.write(json_test)
